In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

import torchvision
import torchvision.transforms as transforms

import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import os
from PIL import Image


In [23]:
class ConvBlock(nn.Module):
  def __init__(self,
               in_channels,
               out_channels,
               kernel_size):
    super().__init__()
    self.block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

  def forward(self, x):
    return self.block(x)

In [35]:
class FlexibleModel(nn.Module):
  def __init__(self,
               img_dim: int,
               num_blocks: int,
               in_channels: int,
               filters: list,
               kernel_sizes: list,
               fc_size: int,
               dropout_rate: float,
               num_classes: int):
    super().__init__()
    filters = [in_channels] + filters
    self.rep_learn = nn.ModuleList([ConvBlock(filters[i], filters[i+1], kernel_sizes[i]) for i in range(num_blocks)])
    self.flatten = nn.Flatten()
    self.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(filters[-1] * (img_dim//2**num_blocks)**2, fc_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(fc_size, num_classes)
    )

  def forward(self, x):
    for block in self.rep_learn:
      x = block(x)
    x = self.flatten(x)
    return self.fc(x)

In [36]:
model = FlexibleModel(224, 3, 3, [16, 32, 64], [3, 3, 3], 256, 0.1, 10)

In [37]:
x = torch.rand((16, 3, 224, 224))
out = model(x)
print(out.shape)

torch.Size([16, 10])


In [31]:
(224//8)**2

784

In [79]:
def extract_tgz(tgz_path, extract_path='.'):
  if not os.path.exists(extract_path):
    os.makedirs(extract_path)
  with tarfile.open(tgz_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)
  print(f"Extracted {tgz_path} to {extract_path}")

In [80]:
def get_all_files(directory_path):
  file_list = []
  for root, _, files in os.walk(directory_path):
    for file in files:
      file_list.append(os.path.join(root, file))
  return file_list

In [132]:
root = "./Images/jpg/"
l = sorted([os.path.join(root,file) for file in list(os.walk(root))[0][2]])
l

['./Images/jpg/image_00001.jpg',
 './Images/jpg/image_00002.jpg',
 './Images/jpg/image_00003.jpg',
 './Images/jpg/image_00004.jpg',
 './Images/jpg/image_00005.jpg',
 './Images/jpg/image_00006.jpg',
 './Images/jpg/image_00007.jpg',
 './Images/jpg/image_00008.jpg',
 './Images/jpg/image_00009.jpg',
 './Images/jpg/image_00010.jpg',
 './Images/jpg/image_00011.jpg',
 './Images/jpg/image_00012.jpg',
 './Images/jpg/image_00013.jpg',
 './Images/jpg/image_00014.jpg',
 './Images/jpg/image_00015.jpg',
 './Images/jpg/image_00016.jpg',
 './Images/jpg/image_00017.jpg',
 './Images/jpg/image_00018.jpg',
 './Images/jpg/image_00019.jpg',
 './Images/jpg/image_00020.jpg',
 './Images/jpg/image_00021.jpg',
 './Images/jpg/image_00022.jpg',
 './Images/jpg/image_00023.jpg',
 './Images/jpg/image_00024.jpg',
 './Images/jpg/image_00025.jpg',
 './Images/jpg/image_00026.jpg',
 './Images/jpg/image_00027.jpg',
 './Images/jpg/image_00028.jpg',
 './Images/jpg/image_00029.jpg',
 './Images/jpg/image_00030.jpg',
 './Images

In [81]:
#mat_data = scipy.io.loadmat('imagelabels.mat')
labels = torch.from_numpy(scipy.io.loadmat('imagelabels.mat')["labels"]).to(torch.long).squeeze()

In [82]:
extract_tgz('102flowers.tgz', './Images')

/tmp/ipython-input-4005182988.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)


Extracted 102flowers.tgz to ./Images


In [83]:
image_dir = "./Images/jpg/"
image_paths = get_all_files(image_dir)
image_paths[:5]

['./Images/jpg/image_07460.jpg',
 './Images/jpg/image_03373.jpg',
 './Images/jpg/image_06906.jpg',
 './Images/jpg/image_00608.jpg',
 './Images/jpg/image_04984.jpg']

In [84]:
img = Image.open(image_paths[0])
type(img)

PIL.JpegImagePlugin.JpegImageFile

In [133]:
class FlowersDataset(Dataset):
  def __init__(self, labels_file, image_dir, transforms=None):
    super().__init__()
    self.labels = torch.from_numpy(scipy.io.loadmat(labels_file)["labels"]).to(torch.long).squeeze()
    self.images = sorted([os.path.join(root,file) for file in list(os.walk(root))[0][2]])
    self.transforms = transforms

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    label = self.labels[index]
    img = Image.open(self.images[index]).convert("RGB")
    if not self.transforms:
      inputs = torchvision.transforms.ToTensor()(img)
    else:
      inputs = self.transforms(img)
    return inputs, label

In [138]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

dataset = FlowersDataset("imagelabels.mat", "./Images/jpg", transform)

In [139]:
dl = DataLoader(dataset, batch_size=64, shuffle=True)

In [140]:
xb, yb = next(iter(dl))
print(xb.shape, yb.shape)

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [3]:
class TestModel(nn.Module):
  def __init__(self, in_channels: int=3, num_classes: int=101):
    super().__init__()
    self.Conv1 = nn.Conv2d(in_channels, 64, 3, 1, 1) #[64, 224, 224]
    self.relu1 = nn.ReLU()
    self.pool = nn.MaxPool2d(2, 2)
    self.Conv2 = nn.Conv2d(64, 64, 3, 1, 1)          #[64, 112, 112]
    self.relu2 = nn.ReLU()
    self.classifier = nn.Sequential(nn.Flatten(),
                                    nn.Linear(64*112*112, num_classes)
                                    )

  def forward(self, x):
    x = self.pool(self.relu1(self.Conv1(x)))
    x = self.relu2(self.Conv2(x))
    return self.classifier(x)

In [4]:
model = TestModel()

In [13]:
for name, param in model.named_parameters():
  print(name, param.shape)

Conv1.weight torch.Size([64, 3, 3, 3])
Conv1.bias torch.Size([64])
Conv2.weight torch.Size([64, 64, 3, 3])
Conv2.bias torch.Size([64])
classifier.1.weight torch.Size([101, 802816])
classifier.1.bias torch.Size([101])


In [23]:
class TensorDatasetWithTransforms(Dataset):
  def __init__(self,
               inputs: torch.tensor,
               labels: torch.tensor,
               input_transform: tuple=None,
               label_transform: tuple=None):
    assert len(inputs) == len(labels)
    self.x = inputs
    self.y = labels
    self.xtransform = input_transform
    self.ytransform = label_transform

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    x, y = self.x[idx], self.y[idx]
    if self.xtransform is not None:
      x = (x - self.xtransform[0]) / self.xtransform[1]
    if self.ytransform is not None:
      y = (y - self.ytransform[0]) / self.ytransform[1]
    return x, y

In [24]:
x = torch.randn((128, 1)) * 5.0
y = torch.rand_like(x)

In [20]:
transform = transforms.Normalize(x.mean(), x.std())

In [25]:
dataset = TensorDatasetWithTransforms(x, y, (x.mean(), x.std()))

In [26]:
x1, y1 = dataset[0]
print(x1, y1)

tensor([2.0958]) tensor([0.4756])


In [27]:
dl = DataLoader(dataset, batch_size=32, shuffle=True)

In [28]:
xb, yb = next(iter(dl))
xb.shape, yb.shape, xb.mean(), xb.std()

(torch.Size([32, 1]), torch.Size([32, 1]), tensor(0.1570), tensor(1.0539))

In [29]:
x[1]

tensor([3.6807])

In [33]:
x.device

device(type='cpu')